In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px
import numpy as np
import hashlib as hl

directory_data = "../data"
athletes = pd.read_csv(f"{directory_data}/athlete_events.csv")
regions = pd.read_csv(f"{directory_data}/noc_regions.csv")

athletes["Name"] = athletes["Name"].astype(str)
hashes = athletes["Name"].apply(lambda x: hl.sha256(x.encode()).hexdigest())
hashes.head()

athletes.insert(1, "Hash values", hashes)
athletes.head()
athletes.drop(columns=["Name"], inplace=True)

merged = pd.merge(athletes, regions, on="NOC", how="outer")

### Australien - Hur har åldrarna på deltagarna sett ut genom åren?


In [39]:
athlete_ages = pd.read_csv(f"{directory_data}/athlete_events_fixed_age.csv")
# anonymisera? 
australia_rows = athlete_ages[(athlete_ages['NOC'] == 'AUS') | (athlete_ages['NOC'] == 'ANZ')]

australia_rows_uniqueID = australia_rows.drop_duplicates(subset=['ID', 'Year'])
australia_rows_uniqueID['Year'] = australia_rows_uniqueID["Year"].astype(int) 
australia_rows_uniqueID.sort_values('Year', inplace=True)

fig = px.histogram(
    australia_rows_uniqueID, 
    x='Age', 
    nbins=35, 
    title="Ages of Australian olympians over the years", 
    animation_frame='Year',
    color='Season'
)

fig.update_layout({
    'yaxis': {'range': [0,100]},
    'xaxis': {'range': [13, 62]}
    }
)
    
#fig['layout'].pop('updatemenus')
fig.show()

C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\1691436907.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\1691436907.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Det blir väldigt hoppigt pga att både sommar- och vinter-OS är med. 
Medelåldern är ca 25 år. 

In [41]:
australia_rows_uniqueID['Age'].describe()

count    5532.000000
mean       25.559472
std         5.838132
min        13.000000
25%        22.000000
50%        25.000000
75%        28.000000
max        62.000000
Name: Age, dtype: float64

# Sport: Rowing

### Hur har åldrarna inom rodd sett ut genom åren?

In [40]:
rowing_rows = merged[merged['Sport']=='Rowing']
rowing_rows_uniqueID = rowing_rows.drop_duplicates(subset=['ID', 'Year'])
rowing_rows_uniqueID['Year'] = rowing_rows_uniqueID['Year'].astype(int)
rowing_rows_uniqueID.sort_values('Year', inplace=True)

fig = px.histogram(
    rowing_rows_uniqueID, 
    x='Age', 
    nbins=42, 
    title="Histogram for Ages in Rowing all countries",
    animation_frame='Year',
    color='Season',
)
    
fig.update_layout({
    'yaxis': {'range': [0,100]},
    'xaxis': {'range': [11, 60]}
    }
)

fig.show()

#jag borde kanske göra en funktion...

C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\325772193.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\325772193.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



I början är det ganska spritt. Från runt 50-talet håller sig medelåldern runt 25 år. 

In [30]:
rowing_rows_uniqueID['Age'].describe()

count    9719.000000
mean       25.283877
std         4.415796
min        11.000000
25%        22.000000
50%        25.000000
75%        28.000000
max        60.000000
Name: Age, dtype: float64

### FRÅGA: Vilka har fått flest medaljer i rodd (topp 10)?

In [19]:
rowing_rows_medals = rowing_rows[rowing_rows['Medal'].notna()]
medals = rowing_rows_medals.groupby(['Team', 'Medal']).size().unstack(fill_value=0) #count medals per Team

medals["Total"] = medals['Gold'] + medals['Silver'] + medals['Bronze']
top10= medals.sort_values(by="Total", ascending=False).head(10).reset_index()

medal_values = ["Bronze", "Silver", "Gold"]
fig = px.bar(top10, x=medal_values, y='Team', color_discrete_map={"Gold":"#FFD700", "Silver": "#C0C0C0", "Bronze": "#CD7F32"}, title="Olympic medals in rowing per 10 top team")

fig.show()

# lägg in slider över tid  - och kanske så att kategorierna ändrar plats! 

#### FRÅGESTÄLLNING: Finns det någon korrelation mellan att få medalj i rodd och längd eller vikt?
Vi gör en correlation pearson plot där vi testar korrelationen mellan medaljer, längd och vikt.


In [20]:
corr_columns = rowing_rows[["Weight", "Height", "Medal"]]

# ofta saknas både weight och height för samma personer. Kanske ta bort de rader där en av dem saknas. 
# så att vi bara kollar korrelation för existerande höjd och vikt med medaljstatus. 
# här får varje person vara med flera gånger! 

# ingen medalj är 0, en medalj är 1 - eventuellt tydligare korrelation med medalj eller ej 
corr_columns["Medal"].fillna(0, inplace=True)
corr_columns["Medal"].mask(corr_columns["Medal"].isin(["Bronze", "Silver", "Gold"]) , other=1, inplace=True)

# innan vi tar bort rader med saknad height elr weight rader har vi 10595 rows
corr_columns.dropna(axis=0, how='any', subset=['Weight', 'Height'], inplace=True)
corr_columns
# kvar har vi 7794 rows 

# create a correlation table with pandas
corr_table = corr_columns.corr(method="pearson")

# set up the correlation plot

fig= px.imshow(
    # pass the correlation matrix,
    corr_table,
    #show the correlation values in each cell,
    text_auto=True, 
    # set the colour scale
    color_continuous_scale="RdYlGn",
    zmin=-1,
    zmax=1, 
    title="Correlation betweeln weight, height and medal in rowing. 0 = no correlation"
 
)
fig.show()

C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\1921462122.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\1921462122.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jubel\AppData\Local\Temp\ipykernel_19940\1921462122.py:9: SettingWithCopyWarning:


A value is trying to be se

### Svar på frågeställning: 

+1 Perfekt positiv korrelation – ökar den ena, ökar den andra exakt lika
0	Ingen linjär korrelation
–1	Perfekt negativ korrelation – ökar den ena, minskar den andra exakt lika

Mellan weight och height finns positiv korrelation (en ökar, andra ökar)

Men det verkar inte finnas nån korrelation mellan Medalj och Weight/Height. På nätet står det att tung och lång är bäst för rodd. Kanske att alla de som tävlar på elit-nivå har den ideala vikten och längden för prestera väl i rodd. Det verkar som en rimlig förklaring till att vi INTE ser en korrelation i denna datan. 
